In [19]:
import numpy as np
import pandas as pd
import binascii
import string
pd.set_option('display.max_rows', None)


In [2]:
# Import the idle data and the command data
idle_main_menu = pd.read_csv('idle_main_menu.csv', delimiter = ",")

laser_startup = pd.read_csv("laser_startup.csv", delimiter = ",")

burst_mode = pd.read_csv("can_log_change_burst_modeV2.csv", delimiter = ",")
idle_main_menu

,Timestamp,Arbitration_id,dlc,data
0,1.757712e+09,0x18820810,5,020a016f09
1,1.757712e+09,0x18820810,3,070101
2,1.757712e+09,0x18820810,8,102045204f464620
3,1.757712e+09,0x18820810,4,00202000
4,1.757712e+09,0x18820810,5,0200007a20
...,...,...,...,...
3746,1.757712e+09,0x19000410,3,0a1fff
3747,1.757712e+09,0x19000410,3,021fff
3748,1.757712e+09,0x19000410,3,071fff
3749,1.757712e+09,0x19000410,3,101fff


In [3]:
# Next is to convert binary to hex to see messages
def hex_to_ascii(hexstr):
    """Convert hex string to ASCII, replace non-printables with '.' """
    try:
        hexstr = str(hexstr).strip()
        if hexstr.lower() in ("nan", "none", ""):
            return ""
        bytes_data = binascii.unhexlify(hexstr)
        return ''.join([chr(b) if chr(b) in string.printable else '.' for b in bytes_data])
    except Exception:
        return ""


In [4]:
# Next is to make a function that adds a column that contains the messages
def decode_can_csv(filename):
    # Load CSV as strings, ignore bad lines
    df = pd.read_csv(filename, dtype=str, header=0, on_bad_lines="skip")

    # Locate the data column
    possible_cols = [c for c in df.columns if "data" in c.lower()]
    if not possible_cols:
        raise ValueError("No 'data' column found in CSV")
    data_col = possible_cols[0]

    # Convert hex payloads to ASCII
    df["ascii"] = df[data_col].astype(str).apply(hex_to_ascii)

    # Print hex alongside ASCII
    decoded = []
    for raw_hex, ascii_text in zip(df[data_col], df["ascii"]):
        if raw_hex and raw_hex.lower() not in ("nan", "none", ""):
            decoded.append((raw_hex, ascii_text))

    return df

In [5]:
filename = "can_log_change_burst_modeV2.csv"  # change to your file
decoded = decode_can_csv(filename)
decoded


,Timestamp,Atribration_id,dlc,data,ascii
0,1757545850.4737952,0x18820810,3,07020f,...
1,1757545850.4739108,0x18820810,8,102020436f6e7469,. Conti
2,1757545850.4740393,0x18820810,7,006e756f757300,.nuous.
3,1757545850.4741473,0x18820810,3,070202,...
4,1757545850.474227,0x18820810,3,101e00,...
...,...,...,...,...,...
782,1757545865.3092425,0x19000410,3,071fff,...
783,1757545865.30963,0x19000410,3,101fff,...
784,1757545865.3160958,0x18820810,1,12,.
785,1757545865.316421,0x19000410,3,121fff,...


In [7]:
# Make a function that finds out how often a pattern repeats
def repeating_pattern(data: list[str]):
    """
    This funciton will take in a column and find how often the data repeats. It will then return the pattern that repeats with the number
    of times it repeats.
    """
    charSet = set() # Create a set 
    left_pointer = 0 # Left hand pointer at the beginning of the string
    result = 0 # Final result count of how oftern it repeats.
    longest_sequence = []

    for i in range(len(data)):
        while data[i] in charSet:
            charSet.remove(data[left_pointer])
            left_pointer += 1
        charSet.add(data[i])

        window_len = i - left_pointer + 1
        if window_len > result:
            result = window_len
            longest_sequence = [data[left_pointer:i + 1]]
        elif window_len == result:
            longest_sequence.append(data[left_pointer:i+1])

    return result, longest_sequence
                                
    

In [8]:
data = idle_main_menu[idle_main_menu['data'] != '12']
repeating = repeating_pattern(list(data['data']))
#repeating

In [9]:
# Next is to see how often things repeat for each device
laser_idle = idle_main_menu[idle_main_menu['Arbitration_id'] == '0x18820810']

controller_idle = idle_main_menu[idle_main_menu['Arbitration_id'] == '0x19000410']

patterns_laser = repeating_pattern(list(laser_idle['data']))
patterns_controller = repeating_pattern(list(controller_idle['data']))

#patterns_laser


In [21]:
laser_startup.head(200)
#After initial messages, the laser sends 30 messages and then the controller responds with another 30 messages. After that 
# it starts its usual cycle of repeated messages.


,Timestamp,Arbitration_id,dlc,data
0,1.757956e+09,0x1803280a,4,00000000
1,1.757956e+09,0x3080a,4,00000000
2,1.757956e+09,0x18820810,1,01
3,1.757956e+09,0x19000410,3,010000
4,1.757956e+09,0x18820810,1,03
5,1.757956e+09,0x18820810,5,020a016f09
6,1.757956e+09,0x18820810,3,070101
7,1.757956e+09,0x18820810,8,102045204f464620
8,1.757956e+09,0x18820810,4,00202000
9,1.757956e+09,0x18820810,5,0200007a20
